In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# modification
# 1. lr_scheduler
# 2. data augument
# 3. more epochs
# 4. loss
# 5. data overlap
# 6. Ensemble
# 7. The 12th layer
# 8. scaler

# 'output_hidden_states':True,"hidden_dropout_prob": 0.0


In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import os
import re
#import cmudictloss = criteon(logits, y)

import torch
from transformers import BertTokenizer, BertModel, BertConfig,BertForSequenceClassification, AdamW
# from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler

#Load Data

In [ ]:
standard = False
eda_ = False

scaler = StandardScaler()
train = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
test=pd.read_csv('../input/commonlitreadabilityprize/test.csv')
train['excerpt'] = train['excerpt'].apply(lambda x: x.replace('\n',''))

array = list(range(len(train))) 
np.random.shuffle(array)
val = train.iloc[array[:100],]
train = train.iloc[array[100:],]

if standard:
    target_ = scaler.fit_transform(np.expand_dims(train['target'].to_numpy(),axis=1))
    train['target'] = np.squeeze(target_)

In [ ]:
# train
text = train.loc[0, 'excerpt']

#Create and generate a word cloud image:
wordcloud = WordCloud().generate(text)

#Display the Generated image:
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
# test
text = test.loc[0, 'excerpt']

#Create and generate a word cloud image:
wordcloud = WordCloud().generate(text)

#Display the Generated image:
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 
			'ours', 'ourselves', 'you', 'your', 'yours', 
			'yourself', 'yourselves', 'he', 'him', 'his', 
			'himself', 'she', 'her', 'hers', 'herself', 
			'it', 'its', 'itself', 'they', 'them', 'their', 
			'theirs', 'themselves', 'what', 'which', 'who', 
			'whom', 'this', 'that', 'these', 'those', 'am', 
			'is', 'are', 'was', 'were', 'be', 'been', 'being', 
			'have', 'has', 'had', 'having', 'do', 'does', 'did',
			'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or',
			'because', 'as', 'until', 'while', 'of', 'at', 
			'by', 'for', 'with', 'about', 'against', 'between',
			'into', 'through', 'during', 'before', 'after', 
			'above', 'below', 'to', 'from', 'up', 'down', 'in',
			'out', 'on', 'off', 'over', 'under', 'again', 
			'further', 'then', 'once', 'here', 'there', 'when', 
			'where', 'why', 'how', 'all', 'any', 'both', 'each', 
			'few', 'more', 'most', 'other', 'some', 'such', 'no', 
			'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 
			'very', 's', 't', 'can', 'will', 'just', 'don', 
			'should', 'now', '']

from nltk.corpus import wordnet 

import re
def get_only_chars(line):

    clean_line = ""

    line = line.replace("’", "")
    line = line.replace("'", "")
    line = line.replace("-", " ") #replace hyphens with spaces
    line = line.replace("\t", " ")
    line = line.replace("\n", " ")
    line = line.lower()

    for char in line:
        if char in 'qwertyuiopasdfghjklzxcvbnm ':
            clean_line += char
        else:
            clean_line += ' '

    clean_line = re.sub(' +',' ',clean_line) #delete extra spaces
    if clean_line[0] == ' ':
        clean_line = clean_line[1:]
    return clean_line


def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words if word not in stop_words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			#print("replaced", random_word, "with", synonym)
			num_replaced += 1
		if num_replaced >= n: #only replace up to n words
			break

	#this is stupid but we need it, trust me
	sentence = ' '.join(new_words)
	new_words = sentence.split(' ')

	return new_words

def get_synonyms(word):
	synonyms = set()
	for syn in wordnet.synsets(word): 
		for l in syn.lemmas(): 
			synonym = l.name().replace("_", " ").replace("-", " ").lower()
			synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
			synonyms.add(synonym) 
	if word in synonyms:
		synonyms.remove(word)
	return list(synonyms)


########################################################################
# Random deletion
# Randomly delete words from the sentence with probability p
########################################################################

def random_deletion(words, p):

	#obviously, if there's only one word, don't delete it
	if len(words) == 1:
		return words

	#randomly delete words with probability p
	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)

	#if you end up deleting all words, just return a random word
	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]

	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################

def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)
	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0
	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words
	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1] 
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################

def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)
	return new_words

def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		random_word = new_words[random.randint(0, len(new_words)-1)]
		synonyms = get_synonyms(random_word)
		counter += 1
		if counter >= 10:
			return
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)
    
    
def eda(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=1):
	
	sentence = get_only_chars(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not '']
	num_words = len(words)
	
	augmented_sentences = []
	num_new_per_technique = int(num_aug/4)+1

# 	#sr
	if (alpha_sr > 0):
		n_sr = max(1, int(alpha_sr*num_words))
		for _ in range(num_new_per_technique):
			a_words = synonym_replacement(words, n_sr)
			augmented_sentences.append(' '.join(a_words))

	#ri
	if (alpha_ri > 0):
		n_ri = max(1, int(alpha_ri*num_words))
		for _ in range(num_new_per_technique):
			a_words = random_insertion(words, n_ri)
			augmented_sentences.append(' '.join(a_words))

	#rs
	if (alpha_rs > 0):
		n_rs = max(1, int(alpha_rs*num_words))
		for _ in range(num_new_per_technique):
			a_words = random_swap(words, n_rs)
			augmented_sentences.append(' '.join(a_words))

	#rd
	if (p_rd > 0):
		for _ in range(num_new_per_technique):
			a_words = random_deletion(words, p_rd)
			augmented_sentences.append(' '.join(a_words))

	augmented_sentences = [get_only_chars(sentence) for sentence in augmented_sentences]
	shuffle(augmented_sentences)

	#trim so that we have the desired number of augmented sentences
	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

	#append the original sentence
	augmented_sentences.append(sentence)

	return augmented_sentences

In [ ]:
num_aug = 1
import random
from random import shuffle
from tqdm import tqdm
random.seed(1)
train_aug = train['excerpt'].tolist()
target_aug = train['target'].tolist()
print("Input shape:", np.array(train_aug).shape)

if eda_:
    for i, line in tqdm(enumerate(train.to_numpy())):
        label = line[4]  # 
        sentence = line[3]
    
        aug_sentences = eda(sentence)
        train_aug.append(aug_sentences)
        target_aug.append(label)

print("Output shape:", np.array(train_aug).shape)

In [ ]:
# train = train.drop(train.index[int(np.where(train['target'].to_numpy()==0)[0])]) 

plt.scatter(train['target'].to_numpy(), train['standard_error'].to_numpy())   # 0.0 - > std 0.64967129

# remove
# incorrect :-(

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_path = '../input/bert-base-uncased'
tokenizer= BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=1)
model

model.to(device)
optim = AdamW(model.parameters(), lr=5e-5, weight_decay=1e-4)      #  params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8,
scheduler = torch.optim.lr_scheduler.StepLR(optim, step_size=3, gamma=0.1, last_epoch=-1)


#create a loader

In [ ]:
class loader(torch.utils.data.Dataset):
    def __init__(self, train_tokens, labels):
        self.train_tokens = train_tokens
        self.labels = labels

    def __getitem__(self, idx):
        
        #train_tokens= tokenizer.batch_encode_plus(self.text_list,max_length=512,padding='longest',truncation=True)
        

        item = {key: torch.tensor(val[idx]) for key, val in self.train_tokens.items()}
        item['labels'] = torch.tensor(self.labels[idx],dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
len(target_aug)

In [ ]:
if eda_:
    train_tokens= tokenizer.batch_encode_plus(train_aug, max_length=512, padding='longest', truncation=True)
    train_dataset = loader(train_tokens, target_aug)
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)  
else:
    train_tokens= tokenizer.batch_encode_plus(train['excerpt'].tolist(), max_length=512, padding='longest', truncation=True)
    train_dataset = loader(train_tokens, train.target.values)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)  # 4-> 32


In [ ]:
val_tokens= tokenizer.batch_encode_plus(val['excerpt'].tolist(),max_length=512,padding='longest',truncation=True)
val_dataset = loader(val_tokens, val.target.values)

val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

test_tokens= tokenizer.batch_encode_plus(test['excerpt'].tolist(),max_length=512,padding='longest',truncation=True)
test['target']=0
test_dataset = loader(test_tokens, test.target.values)

test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

In [ ]:
from transformers import AutoModel,AutoConfig, AutoTokenizer,get_cosine_schedule_with_warmup

lr_scheduler = get_cosine_schedule_with_warmup(optim,num_warmup_steps=0,num_training_steps=  int(len(train)/10))   # 10 *
   


In [ ]:
total_test_preds = []
val_rmse = []
from tqdm import tqdm
for epoch in range(3):    #3
    for batch in tqdm(train_loader):
   
#         optim.zero_grad()  ->
        optim.zero_grad(set_to_none=True) 
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device).reshape(attention_mask.shape[0],-1)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]  # MseLossBackward
#         print("loss:", outputs[0], flush=True)
        # sqrt
        # return torch.sqrt(nn.MSELoss()(outputs,targets))
        loss.backward()
        optim.step()
        if lr_scheduler:
            lr_scheduler.step()
    
    
        # validation
    total_preds = []
    # iterate over batches
    for step, batch in enumerate(val_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device).reshape(attention_mask.shape[0],-1)
        with torch.no_grad():

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)        
            
            preds = outputs[1].detach().cpu().numpy()
            if standard:
                preds = scaler.inverse_transform(preds)
                
            total_preds.extend(preds-labels.detach().cpu().numpy())

    val_rmse.append(np.sqrt(np.mean(np.square(total_preds))))
    print("Epoch ", epoch, " RMSE: ", np.sqrt(np.mean(np.square(total_preds))))
    # test
    total_preds = []
    for step, batch in enumerate(test_loader):
        print(step)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device).reshape(attention_mask.shape[0],-1)
        with torch.no_grad():

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)        

            preds = outputs[1].detach().cpu().numpy()
            if standard:
                    preds = scaler.inverse_transform(preds)

            total_test_preds.append(preds)
    print(total_test_preds)

In [ ]:
total_preds = []
for step, batch in enumerate(val_loader):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device).reshape(attention_mask.shape[0],-1)
    with torch.no_grad():

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)        
        loss = outputs[0].detach().cpu().numpy()
        preds = outputs[1].detach().cpu().numpy()
        if standard:
                preds = scaler.inverse_transform(preds)

        total_preds.extend(preds-labels.detach().cpu().numpy())


print("RMSE: ", np.sqrt(np.mean(np.square(total_preds))))

In [ ]:
print(np.array(total_test_preds))
val_rmse

In [ ]:
total_preds = []
for step, batch in enumerate(test_loader):
    print(step)
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device).reshape(attention_mask.shape[0],-1)
    with torch.no_grad():

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)        

        preds = outputs[1].detach().cpu().numpy()
        if standard:
                preds = scaler.inverse_transform(preds)

        total_preds.append(preds)
print(total_preds)

In [ ]:
# total_test_preds = np.array(total_test_preds)
# total_preds = total_test_preds[np.where(val_rmse==np.min(val_rmse))]
# print( np.concatenate(total_preds, axis=0))     

sample = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')
sample['target'] = np.concatenate(total_preds, axis=0)

sample.to_csv('/kaggle/working/submission.csv', index=False)
print(sample)